In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Embedding, Dropout, Input, Concatenate
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
third_model = load_model('/content/drive/MyDrive/Election Project/third_model.h5')

In [4]:
df_2018 = pd.read_csv('/content/drive/MyDrive/Election Project/Complete 2018.csv')
df_2018.isnull().sum()

constituency_number     0
consitituencey_name     0
province                0
candidate_name          0
candidate_party         0
candidate_votes         0
candidate_share        16
outcome                 1
Total NA Votes          0
dtype: int64

In [5]:
x=df_2018['constituency_number'].unique()
len(x)

272

In [6]:
df_2018['candidate_share'].fillna(df_2018['candidate_share'].mean(), inplace=True)
df_2018['outcome'].fillna(0, inplace=True)

In [25]:
df_2018.head()

,constituency_number,consitituencey_name,province,candidate_name,candidate_party,candidate_votes,candidate_share,outcome,Total NA Votes
0,NA-1,Chitral,Khyber Paktunkhwa (NWFP),Molana Abdul Akbar Chitrali,Muttahidda Majlis-e-Amal Pakistan,48616,0.369195,win,131681
1,NA-1,Chitral,Khyber Paktunkhwa (NWFP),Abdul Latif,Pakistan Tehreek-e-Insaf,40002,0.303780,loss,131681
2,NA-1,Chitral,Khyber Paktunkhwa (NWFP),Saleem Khan,Pakistan Peoples Party (Shaheed Bhutto),32635,0.247834,loss,131681
3,NA-1,Chitral,Khyber Paktunkhwa (NWFP),Iftikhar Ud Din,Pakistan Muslim League (Nawaz),2101,0.015955,loss,131681
4,NA-1,Chitral,Khyber Paktunkhwa (NWFP),Eid Ul Hussain,Awami National Party,3613,0.027438,loss,131681


In [7]:
df_2018.isnull().sum()

constituency_number    0
consitituencey_name    0
province               0
candidate_name         0
candidate_party        0
candidate_votes        0
candidate_share        0
outcome                0
Total NA Votes         0
dtype: int64

In [8]:
len(df_2018['constituency_number'].unique())

272

In [9]:
desire_col = ['constituency_number', 'province', 'candidate_name','candidate_party','candidate_votes','candidate_share','outcome']
data = df_2018[desire_col]

In [10]:
max_candidate_name_length = 40
max_candidate_party_length = 40

In [11]:
def tokenize_and_pad(data,max_len):
  sequences = pad_sequences(data, maxlen=max_len, padding='post')
  print(sequences)
  return sequences.tolist()

In [12]:
data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())

# Extract unique constituencies
constituencies = data['constituency_number'].unique()

<ipython-input-12-df51f43c78f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
<ipython-input-12-df51f43c78f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())


In [13]:
all_features, all_labels = [], []

for constituency_number in constituencies:
    group = data[data['constituency_number'] == constituency_number]

    features = group[['province', 'candidate_name', 'candidate_party', 'candidate_votes', 'candidate_share']]
    labels = group['outcome'].astype('category').cat.codes

    all_features.append(features)
    all_labels.append(labels)

processed_data = pd.concat(all_features, ignore_index=True)
processed_data['outcome'] = pd.concat(all_labels, ignore_index=True)

In [14]:
# Convert text columns to sequences
tok = Tokenizer(num_words=10000)
tok.fit_on_texts(processed_data['candidate_name'].astype(str))
processed_data['candidate_name_padded'] = tok.texts_to_sequences(processed_data['candidate_name'].astype(str))

tok.fit_on_texts(processed_data['candidate_party'].astype(str))
processed_data['candidate_party_padded'] = tok.texts_to_sequences(processed_data['candidate_party'].astype(str))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(processed_data[['province', 'candidate_name_padded', 'candidate_party_padded', 'candidate_votes', 'candidate_share']], processed_data['outcome'], test_size=0.2, random_state=42)

In [26]:
from keras.layers import Bidirectional, Reshape

model4 = Sequential()
model4.add(third_model)
model4.add(Reshape((-1, 1)))
model4.add(LSTM(64, return_sequences=True))
model4.add(Dropout(0.2))
model4.add(LSTM(32, return_sequences=True))
model4.add(LSTM(32))
model4.add(Dense(16, activation='relu'))
model4.add(Dense(1, activation='tanh'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
X_train_padded = pad_sequences(X_train['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')

# Assuming max_candidate_name_length is the length of your padded sequences
timesteps = max_candidate_name_length
features = 1  # Assuming each element in the sequence is a single feature

X_train_padded = X_train_padded.reshape((X_train_padded.shape[0], timesteps, features))

# Train the model
model4.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
65/65 [==============================] - 37s 207ms/step - loss: 0.3613 - accuracy: 0.9088 - val_loss: 0.2612 - val_accuracy: 0.9283
Epoch 2/10
65/65 [==============================] - 8s 122ms/step - loss: 0.3122 - accuracy: 0.9088 - val_loss: 0.2584 - val_accuracy: 0.9283
Epoch 3/10
65/65 [==============================] - 6s 98ms/step - loss: 0.3075 - accuracy: 0.9088 - val_loss: 0.2593 - val_accuracy: 0.9283
Epoch 4/10
65/65 [==============================] - 8s 118ms/step - loss: 0.3067 - accuracy: 0.9088 - val_loss: 0.2606 - val_accuracy: 0.9283
Epoch 5/10
65/65 [==============================] - 6s 90ms/step - loss: 0.3066 - accuracy: 0.9088 - val_loss: 0.2610 - val_accuracy: 0.9283
Epoch 6/10
65/65 [==============================] - 7s 115ms/step - loss: 0.3065 - accuracy: 0.9088 - val_loss: 0.2611 - val_accuracy: 0.9283
Epoch 7/10
65/65 [==============================] - 6s 98ms/step - loss: 0.3065 - accuracy: 0.9088 - val_loss: 0.2604 - val_accuracy: 0.9283
Epoch 8/

In [18]:
model4.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 1)                 1427043   
                                                                 
 reshape (Reshape)           (None, 1, 1)              0         
                                                                 
 lstm (LSTM)                 (None, 1, 64)             16896     
                                                                 
 dropout (Dropout)           (None, 1, 64)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 32)             12416     
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 16)                5

In [ ]:
#model4.save('/content/drive/MyDrive/Election Project/fourth_model.h5')

In [19]:
X_test_padded = pad_sequences(X_test['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')
X_test_padded = X_test_padded.reshape((X_test_padded.shape[0], timesteps, features))

# Evaluate the model on the test set
evaluation = model4.evaluate(X_test_padded, y_test)
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

21/21 [==============================] - 0s 21ms/step - loss: 0.3134 - accuracy: 0.9054
Test Loss: 0.3133753538131714
Test Accuracy: 0.9054263830184937


In [21]:
predictions_2018 = model4.predict(X_test_padded)
predictions_2018

21/21 [==============================] - 1s 40ms/step


array([[0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.08669613],
       [0.086